<a href="https://colab.research.google.com/github/jmarrietar/recommenders/blob/jmarrietar%2Franking/basic_ranking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Plan de accion:
---------------

    - 1. Sacar metricas con el utils de recommenders.
    - 2. Intentar utilizar el load de recommenders aca. (Opcional: - aunque ayudaria pal de Pytorch tener esto dominado - Hacer otra version)
    - 3. Cuando ya tenga esto domindado tf-recommenders migrar implementacion a Pytorch.
    - 4. Hacer Notebook para recommenders.
"""

# Recommending movies: ranking

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/recommenders/examples/basic_ranking"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/recommenders/blob/main/docs/examples/basic_ranking.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/recommenders/blob/main/docs/examples/basic_ranking.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/recommenders/docs/examples/basic_ranking.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>



Real-world recommender systems are often composed of two stages:

1. The retrieval stage is responsible for selecting an initial set of hundreds of candidates from all possible candidates. The main objective of this model is to efficiently weed out all candidates that the user is not interested in. Because the retrieval model may be dealing with millions of candidates, it has to be computationally efficient.
2. The ranking stage takes the outputs of the retrieval model and fine-tunes them to select the best possible handful of recommendations. Its task is to narrow down the set of items the user may be interested in to a shortlist of likely candidates.

We're going to focus on the second stage, ranking. If you are interested in the retrieval stage, have a look at our [retrieval](basic_retrieval) tutorial.

In this tutorial, we're going to:

1. Get our data and split it into a training and test set.
2. Implement a ranking model.
3. Fit and evaluate it.


## Imports


Let's first get our imports out of the way.

In [ ]:
#!pip install -q tensorflow-recommenders
#!pip install -q --upgrade tensorflow-datasets

In [ ]:
#pip install tensorflow==2.15


In [1]:
!pip install recommenders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 7.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of notebook to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.3/355.3 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.4/25.4 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 82.2 MB/s eta 0:00:00
   ━━━

In [ ]:
"""
TODO:
    - Instalar recommenders aqui a ver si puedo utilizarle las herramientas de medicion!.
"""

In [2]:
from recommenders.evaluation.python_evaluation import (
    map,
    ndcg_at_k,
    precision_at_k,
    recall_at_k,
    rmse,
    mae,
    logloss,
    rsquared,
    exp_var
)

In [4]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [5]:
#import tensorflow_recommenders as tfrs

In [6]:
print(tf.__version__)

2.18.0


In [7]:
# ---------------
#    base.py
# ---------------

# Copyright 2024 The TensorFlow Recommenders Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# lint-as: python3

"""Base model."""

import tensorflow as tf


class Model(tf.keras.Model):
  """Base model for TFRS models.

  Many recommender models are relatively complex, and do not neatly fit into
  supervised or unsupervised paradigms. This base class makes it easy to
  define custom training and test losses for such complex models.

  This is done by asking the user to implement the following methods:
  - `__init__` to set up your model. Variable, task, loss, and metric
    initialization should go here.
  - `compute_loss` to define the training loss. The method takes as input the
    raw features passed into the model, and returns a loss tensor for training.
    As part of doing so, it should also update the model's metrics.
  - [Optional] `call` to define how the model computes its predictions. This
    is not always necessary: for example, two-tower retrieval models have two
    well-defined submodels whose `call` methods are normally used directly.

  Note that this base class is a thin conveniece wrapper for tf.keras.Model, and
  equivalent functionality can easily be achieved by overriding the `train_step`
  and `test_step` methods of a plain Keras model. Doing so also makes it easy
  to build even more complex training mechanisms, such as the use of
  different optimizers for different variables, or manipulating gradients.

  Keras has an excellent tutorial on how to
  do this [here](
  https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit).
  """

  def compute_loss(self, inputs, training: bool = False) -> tf.Tensor:  # pytype: disable=signature-mismatch  # overriding-parameter-count-checks
    """Defines the loss function.

    Args:
      inputs: A data structure of tensors: raw inputs to the model. These will
        usually contain labels and weights as well as features.
      training: Whether the model is in training mode.

    Returns:
      Loss tensor.
    """

    raise NotImplementedError(
        "Implementers must implement the `compute_loss` method.")

  def train_step(self, inputs):
    """Custom train step using the `compute_loss` method."""

    with tf.GradientTape() as tape:
      loss = self.compute_loss(inputs, training=True)

      # Handle regularization losses as well.
      regularization_loss = tf.reduce_sum(
          [tf.reduce_sum(loss) for loss in self.losses]
      )

      total_loss = loss + regularization_loss

    gradients = tape.gradient(total_loss, self.trainable_variables)
    self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

    metrics = {metric.name: metric.result() for metric in self.metrics}
    metrics["loss"] = loss
    metrics["regularization_loss"] = regularization_loss
    metrics["total_loss"] = total_loss

    return metrics

  def test_step(self, inputs):
    """Custom test step using the `compute_loss` method."""

    loss = self.compute_loss(inputs, training=False)

    # Handle regularization losses as well.
    regularization_loss = tf.reduce_sum(
        [tf.reduce_sum(loss) for loss in self.losses]
    )

    total_loss = loss + regularization_loss

    metrics = {metric.name: metric.result() for metric in self.metrics}
    metrics["loss"] = loss
    metrics["regularization_loss"] = regularization_loss
    metrics["total_loss"] = total_loss

    return metrics

In [9]:
# ---------------
#    ranking.py
# ---------------

# Copyright 2024 The TensorFlow Recommenders Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Lint-as: python3

"""A ranking task."""

from typing import List, Optional, Text

import tensorflow as tf


class Ranking(tf.keras.layers.Layer):
  """A ranking task.

  Recommender systems are often composed of two components:
  - a retrieval model, retrieving O(thousands) candidates from a corpus of
    O(millions) candidates.
  - a ranker model, scoring the candidates retrieved by the retrieval model to
    return a ranked shortlist of a few dozen candidates.

  This task helps with building ranker models. Usually, these will involve
  predicting signals such as clicks, cart additions, likes, ratings, and
  purchases.
  """

  def __init__(
      self,
      loss: Optional[tf.keras.losses.Loss] = None,
      metrics: Optional[List[tf.keras.metrics.Metric]] = None,
      prediction_metrics: Optional[List[tf.keras.metrics.Metric]] = None,
      label_metrics: Optional[List[tf.keras.metrics.Metric]] = None,
      loss_metrics: Optional[List[tf.keras.metrics.Metric]] = None,
      name: Optional[Text] = None) -> None:
    """Initializes the task.

    Args:
      loss: Loss function. Defaults to BinaryCrossentropy.
      metrics: List of Keras metrics to be evaluated.
      prediction_metrics: List of Keras metrics used to summarize the
        predictions.
      label_metrics: List of Keras metrics used to summarize the labels.
      loss_metrics: List of Keras metrics used to summarize the loss.
      name: Optional task name.
    """

    super().__init__(name=name)

    self._loss = (
        loss if loss is not None else tf.keras.losses.BinaryCrossentropy())
    self._ranking_metrics = metrics or []
    self._prediction_metrics = prediction_metrics or []
    self._label_metrics = label_metrics or []
    self._loss_metrics = loss_metrics or []

  def call(self,
           labels: tf.Tensor,
           predictions: tf.Tensor,
           sample_weight: Optional[tf.Tensor] = None,
           training: bool = False,
           compute_metrics: bool = True) -> tf.Tensor:
    """Computes the task loss and metrics.

    Args:
      labels: Tensor of labels.
      predictions: Tensor of predictions.
      sample_weight: Tensor of sample weights.
      training: Indicator whether training or test loss is being computed.
      compute_metrics: Whether to compute metrics. Set this to False
        during training for faster training.

    Returns:
      loss: Tensor of loss values.
    """

    loss = self._loss(
        y_true=labels, y_pred=predictions, sample_weight=sample_weight)

    if not compute_metrics:
      return loss

    update_ops = []

    for metric in self._ranking_metrics:
      update_ops.append(metric.update_state(
          y_true=labels, y_pred=predictions, sample_weight=sample_weight))

    for metric in self._prediction_metrics:
      update_ops.append(
          metric.update_state(predictions, sample_weight=sample_weight))

    for metric in self._label_metrics:
      update_ops.append(
          metric.update_state(labels, sample_weight=sample_weight))

    for metric in self._loss_metrics:
      update_ops.append(
          metric.update_state(loss)
      )  # Loss is a scalar here which is already weighted sum

    # Custom metrics may not return update ops, unlike built-in
    # Keras metrics.
    update_ops = [x for x in update_ops if x is not None]

    with tf.control_dependencies(update_ops):
      return tf.identity(loss)


## Preparing the dataset

We're going to use the same data as the [retrieval](basic_retrieval) tutorial. This time, we're also going to keep the ratings: these are the objectives we are trying to predict.

In [19]:
ratings = tfds.load("movielens/100k-ratings", split="train")

In [21]:
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "user_rating": x["user_rating"]
})

In [12]:
type(ratings)

tensorflow.python.data.ops.map_op._MapDataset

In [13]:
ratings

<_MapDataset element_spec={'movie_title': TensorSpec(shape=(), dtype=tf.string, name=None), 'user_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'user_rating': TensorSpec(shape=(), dtype=tf.float32, name=None)}>

As before, we'll split the data by putting 80% of the ratings in the train set, and 20% in the test set.

In [22]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

In [15]:
shuffled

<_ShuffleDataset element_spec={'movie_title': TensorSpec(shape=(), dtype=tf.string, name=None), 'user_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'user_rating': TensorSpec(shape=(), dtype=tf.float32, name=None)}>

In [23]:
train = shuffled.take(80_000)

In [24]:
test = shuffled.skip(80_000).take(20_000)

In [26]:
test

<_TakeDataset element_spec={'movie_title': TensorSpec(shape=(), dtype=tf.string, name=None), 'user_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'user_rating': TensorSpec(shape=(), dtype=tf.float32, name=None)}>

Let's also figure out unique user ids and movie titles present in the data.

This is important because we need to be able to map the raw values of our categorical features to embedding vectors in our models. To do that, we need a vocabulary that maps a raw feature value to an integer in a contiguous range: this allows us to look up the corresponding embeddings in our embedding tables.

In [27]:
movie_titles = ratings.batch(1_000_000).map(lambda x: x["movie_title"])

In [28]:
movie_titles

<_MapDataset element_spec=TensorSpec(shape=(None,), dtype=tf.string, name=None)>

In [29]:
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

In [30]:
unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))

In [31]:
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [ ]:
len(unique_movie_titles)

1664

In [ ]:
len(unique_user_ids)

943

## Implementing a model

### Architecture

Ranking models do not face the same efficiency constraints as retrieval models do, and so we have a little bit more freedom in our choice of architectures.

A model composed of multiple stacked dense layers is a relatively common architecture for ranking tasks. We can implement it as follows:

In [ ]:
class RankingModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    embedding_dimension = 32

    # Compute embeddings for users.
    self.user_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # Compute embeddings for movies.
    self.movie_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
    ])

    # Compute predictions.
    self.ratings = tf.keras.Sequential([
      # Learn multiple dense layers.
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      # Make rating predictions in the final layer.
      tf.keras.layers.Dense(1)
  ])

  def call(self, inputs):

    user_id, movie_title = inputs

    user_embedding = self.user_embeddings(user_id)
    movie_embedding = self.movie_embeddings(movie_title)

    return self.ratings(tf.concat([user_embedding, movie_embedding], axis=1))

In [ ]:
# TODO: DIBUJAR EL MODELO

This model takes user ids and movie titles, and outputs a predicted rating:

In [ ]:
RankingModel()((["42"], ["One Flew Over the Cuckoo's Nest (1975)"]))

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-0.00655902]], dtype=float32)>

### Loss and metrics

The next component is the loss used to train our model. TFRS has several loss layers and tasks to make this easy.

In this instance, we'll make use of the `Ranking` task object: a convenience wrapper that bundles together the loss function and metric computation.

We'll use it together with the `MeanSquaredError` Keras loss in order to predict the ratings.

In [ ]:
"""
TODO:
- Mirar que onda con ese tfrs.task.Ranking (Creo que tambien lo usamos), me gustaria saber que hace.

"""

'\nTODO:\n- Mirar que onda con ese tfrs.task.Ranking (Creo que tambien lo usamos), me gustaria saber que hace.\n\n'

In [ ]:
"""
task = tfrs.tasks.Ranking(
  loss = tf.keras.losses.MeanSquaredError(),
  metrics=[tf.keras.metrics.RootMeanSquaredError()]
)
"""

'\ntask = tfrs.tasks.Ranking(\n  loss = tf.keras.losses.MeanSquaredError(),\n  metrics=[tf.keras.metrics.RootMeanSquaredError()]\n)\n'

In [ ]:
"""
task = Ranking(
  loss = tf.keras.losses.MeanSquaredError(),
  metrics=[tf.keras.metrics.RootMeanSquaredError()]
)
"""

'\ntask = Ranking(\n  loss = tf.keras.losses.MeanSquaredError(),\n  metrics=[tf.keras.metrics.RootMeanSquaredError()]\n)\n'

The task itself is a Keras layer that takes true and predicted as arguments, and returns the computed loss. We'll use that to implement the model's training loop.

### The full model

We can now put it all together into a model. TFRS exposes a base model class (`tfrs.models.Model`) which streamlines bulding models: all we need to do is to set up the components in the `__init__` method, and implement the `compute_loss` method, taking in the raw features and returning a loss value.

The base model will then take care of creating the appropriate training loop to fit our model.

In [ ]:
#class MovielensModel(tfrs.models.Model):

class MovielensModel(Model):

  def __init__(self):
    super().__init__()
    self.ranking_model: tf.keras.Model = RankingModel()
    """
    self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
      loss = tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )
    """
    self.task: tf.keras.layers.Layer = Ranking(
      loss = tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )

  def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
    return self.ranking_model(
        (features["user_id"], features["movie_title"]))

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    labels = features.pop("user_rating")

    rating_predictions = self(features)

    # The task computes the loss and the metrics.
    return self.task(labels=labels, predictions=rating_predictions)

## Fitting and evaluating

After defining the model, we can use standard Keras fitting and evaluation routines to fit and evaluate the model.

Let's first instantiate the model.

In [ ]:
model = MovielensModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

Then shuffle, batch, and cache the training and evaluation data.

In [ ]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

Then train the  model:

In [ ]:
model.fit(cached_train, epochs=3)

Epoch 1/3
10/10 [==============================] - 7s 150ms/step - root_mean_squared_error: 2.0881 - loss: 4.0167 - regularization_loss: 0.0000e+00 - total_loss: 4.0167
Epoch 2/3
10/10 [==============================] - 1s 54ms/step - root_mean_squared_error: 1.1272 - loss: 1.2724 - regularization_loss: 0.0000e+00 - total_loss: 1.2724
Epoch 3/3
10/10 [==============================] - 1s 53ms/step - root_mean_squared_error: 1.1188 - loss: 1.2531 - regularization_loss: 0.0000e+00 - total_loss: 1.2531


As the model trains, the loss is falling and the RMSE metric is improving.

Finally, we can evaluate our model on the test set:

In [ ]:
model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 4s 31ms/step - root_mean_squared_error: 1.1109 - loss: 1.2295 - regularization_loss: 0.0000e+00 - total_loss: 1.2295


{'root_mean_squared_error': 1.1108818054199219,
 'loss': 1.209473729133606,
 'regularization_loss': 0.0,
 'total_loss': 1.209473729133606}

The lower the RMSE metric, the more accurate our model is at predicting ratings.

## Testing the ranking model

Now we can test the ranking model by computing predictions for a set of movies and then rank these movies based on the predictions:


In [ ]:
test_ratings = {}
test_movie_titles = ["M*A*S*H (1970)", "Dances with Wolves (1990)", "Speed (1994)"]
for movie_title in test_movie_titles:
  test_ratings[movie_title] = model({
      "user_id": np.array(["50"]),
      "movie_title": np.array([movie_title])
  })

print("Ratings:")
for title, score in sorted(test_ratings.items(), key=lambda x: x[1], reverse=True):
  print(f"{title}: {score}")

Ratings:
M*A*S*H (1970): [[3.4290922]]
Dances with Wolves (1990): [[3.409481]]
Speed (1994): [[3.3466034]]


## TODO: Evaluar con metricas de recommenders.

Seguir mi trabajo aqui


TODO: - Para todos los usuarios en test, hacerle prediccion con todas las movies y rankearlas.

---



In [ ]:
"""
TODO: - HACER PREDICCION A USUARIOS DE TEST
    - Para todos los usuarios en test, hacerle prediccion con todas las movies y Rankearlas.
"""

In [ ]:
"""
Pensamiento:
- Tengo que llegar a tener algo como recommenders de tipo

	userID	itemID	prediction
0	1	433	2.910697
1	1	204	2.906224
2	1	403	2.906136
3	1	174	2.870639
4	1	70	2.863253
5	1	4	2.854544
6	1	98	2.845376
7	1	144	2.817695
8	1	423	2.816989
9	1	228	2.813345

"""

In [ ]:
"""
    TODO: - Objetivo: Sacarle METRCAS a esto. (Utilizando recommmenders utils)
        -  Utilizar las herramientas de medicion de recommenders para SACARLE METRICAS a esto!.
"""

## Exporting for serving

The model can be easily exported for serving:


In [ ]:
tf.saved_model.save(model, "export")

We can now load it back and perform predictions:

In [ ]:
loaded = tf.saved_model.load("export")

loaded({"user_id": np.array(["42"]), "movie_title": ["Speed (1994)"]}).numpy()

array([[3.400573]], dtype=float32)

## Convert the model to TensorFLow Lite

Although TensorFlow Recommenders is primarily designed to perform server-side recommendations, you can still convert the trained ranking model to TensorFLow Lite and run it on-device (for better user privacy privacy and lower latency).


In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model("export")
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)

544480

Once the model is converted, you can run it like regular TensorFlow Lite models. Please check out [TensorFlow Lite documentation](https://www.tensorflow.org/lite/guide/inference) to learn more.

In [ ]:
interpreter = tf.lite.Interpreter(model_path="converted_model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model.
if input_details[0]["name"] == "serving_default_movie_title:0":
  interpreter.set_tensor(input_details[0]["index"], np.array(["Speed (1994)"]))
  interpreter.set_tensor(input_details[1]["index"], np.array(["42"]))
else:
  interpreter.set_tensor(input_details[0]["index"], np.array(["42"]))
  interpreter.set_tensor(input_details[1]["index"], np.array(["Speed (1994)"]))

interpreter.invoke()

rating = interpreter.get_tensor(output_details[0]['index'])
print(rating)

## Next steps

The model above gives us a decent start towards building a ranking system.

Of course, making a practical ranking system requires much more effort.

In most cases, a ranking model can be substantially improved by using more features rather than just user and candidate identifiers. To see how to do that, have a look at the [side features](featurization) tutorial.

A careful understanding of the objectives worth optimizing is also necessary. To get started on building a recommender that optimizes multiple objectives, have a look at our [multitask](multitask) tutorial.